In [1]:
!nvidia-smi

Sat Dec 11 03:04:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   49C    P0    27W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install lightweight-gan pytorch-fid

In [13]:
import os
import torch
import boto3
print(torch.cuda.is_available())

True


In [14]:
def edit_file(filename, original, replacement):
    with open(filename, 'r') as f:
        py_data = f.read()
        f.close()
        
    py_data = py_data.replace(original, replacement)
    
    with open(filename, 'w') as f:
        f.write(py_data)
        f.close()
    print('Done')

In [15]:
file_to_edit = '/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/PIL/ImageFile.py'
edit_file(
    file_to_edit,
    'LOAD_TRUNCATED_IMAGES = False', 
    'LOAD_TRUNCATED_IMAGES = True'
)

Done


In [16]:
with open(file_to_edit, 'r') as f:
    py_data = f.read()
    f.close()
    
py_data = py_data.replace('LOAD_TRUNCATED_IMAGES = False', 'LOAD_TRUNCATED_IMAGES = True')
with open(file_to_edit, 'w') as f:
    f.write(py_data)
    f.close()

In [18]:
!aws s3 sync s3://sol-ar-system/data ./data

download: s3://sol-ar-system/data/raw/100_100.jpg to data/raw/100_100.jpg             
download: s3://sol-ar-system/data/raw/1.png to data/raw/1.png                       
download: s3://sol-ar-system/data/raw/0.jpg to data/raw/0.jpg                       
download: s3://sol-ar-system/data/raw/0_0.jpg to data/raw/0_0.jpg                   
download: s3://sol-ar-system/data/raw/103_103.jpg to data/raw/103_103.jpg          
download: s3://sol-ar-system/data/raw/1.jpg to data/raw/1.jpg                      
download: s3://sol-ar-system/data/raw/103.jpg to data/raw/103.jpg                  
download: s3://sol-ar-system/data/raw/107.jpg to data/raw/107.jpg                  
download: s3://sol-ar-system/data/raw/104_104.jpg to data/raw/104_104.jpg          
download: s3://sol-ar-system/data/raw/101_101.jpg to data/raw/101_101.jpg           
download: s3://sol-ar-system/data/raw/102_102.jpg to data/raw/102_102.jpg           
download: s3://sol-ar-system/data/raw/109_109.jpg to data/raw/109_10

In [19]:
!aws s3 sync s3://sol-ar-system/models ./models

In [17]:
file_to_edit = '/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/lightweight_gan/lightweight_gan.py'
edit_file(
    file_to_edit,
    'from kornia import filter2d', 
    'from kornia.filters import filter2d'
)

Done


In [18]:
file_to_edit = '/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/lightweight_gan/__init__.py'

edit_file(
    file_to_edit,
    'from kornia import filter2d', 
    'from kornia.filters import filter2d'
)

Done


In [ ]:
!lightweight_gan \
    --data ./data/raw \
    --name sol-ar-system \
    --results_dir ./data/results \
    --models_dir ./models \
    --image_size 512 \
    --aug-prob 0.25 \
    --aug-types [translation,cutout,color] \
    --dataset-aug-prob .5 \
    --trunc-psi 1 \
    --cl-reg

sol-ar-system<./data/raw>:   0%|                     | 0/150000 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
sol-ar-system<./data/raw>:  16%|▋   | 24571/150000 [9:17:20<47:25:04,  1.36s/it]

In [37]:
torch.cuda.empty_cache()

In [21]:
!rm -rf models/sol-ar-system/